In [ ]:
import pandas as pd

df = pd.read_json('data\General_validation_text.jsonl', lines = True)
df.drop(columns = ['id'], inplace = True)
df.rename(columns = {'article' : 'input_text', 'highlights': "target_text"}, inplace = True)
df.to_json('data\General_validation_text.jsonl', orient = 'records',lines = True)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
import torch

#load long-T5
model_name = "google/long-t5-tglobal-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data = pd.read_json('data/General_train_text_chunk_1.jsonl', lines = True)
dataset = Dataset.from_dict(data)

#Tokenize
def preprocess(example):
    input_text = ["summarize: " + x for x in example["input_text"]]
    inputs = tokenizer(input_text, padding = "max_length", max_length = 4096, truncation = True)
    targets = tokenizer(example["target_text"], padding = "max_length", max_length = 256, truncation = True)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched = True, remove_columns = dataset.column_names)

training_args = TrainingArguments(
    output_dir="./longt5-summarizer",            
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_strategy="epoch",               
    logging_dir="./logs",                
    fp16=torch.cuda.is_available(),      
    save_total_limit=2
)

#Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

In [ ]:
trainer.train()

In [17]:
import pandas as pd

df = pd.read_json('data\General_train_text_chunk_1.jsonl', lines = True)
your_article_text = df.at[0, 'input_text']
ans = df.at[0, 'target_text']

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/long-t5-tglobal-base"
tokenizer = AutoTokenizer.from_pretrained("local-t5/tokenizer")
model = AutoModelForSeq2SeqLM.from_pretrained("local-t5/model")

In [18]:
text = "summarize: " + your_article_text
inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=256, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)
print(ans)

C:\Users\hp333\AppData\Roaming\Python\Python311\site-packages\transformers\modeling_utils.py:1006: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Daniel radcliffe as harry potter in "harry potter and the order of the phoenix" to the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
Harry potter star daniel radcliffe gets £20m fortune as he turns 18 monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five potter films have been held in trust fund .
